# Aplicando e Definindo o Modelo

As camadas ``GlobalAveragePooling2D`` e ``Flatten`` servem como "pontes" entre as camadas convolucionais (que trabalham com dados espaciais, como imagens) e as camadas totalmente conectadas (que esperam dados em um formato linear). A principal diferença é como elas processam os dados, o que impacta o tamanho do modelo e sua capacidade de generalização. 

Explicação diferença entre ``GlobalAveragePooling2D`` e ``Flatten``:

https://chatgpt.com/share/693c57c4-343c-8009-a61e-e898192bf8bf

In [ ]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(32,32,3)),

    layers.Resizing(32,32),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    # filtros
    # o filtro precisa treinar
    layers.Conv2D(32, activation='relu', kernel_size=(3,3), strides=(1,1), padding='same'),

    # reduz o tamanho da imagem pegando apenas o valor máximo dentro de grids, evita o overfitting
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, activation='relu', kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D(),

    layers.GlobalAveragePooling2D(),

    # ReLU pq ignora valores negativos
    layers.Dense(
        32,
        activations.relu,
        kernel_initializer = initializers.RandomNormal()),

    # desativa os neurônios que estão aprendendo demais (opcional) 
    # layers.Dropout()

    # softmax pq ele escolhe uma classe entre várias. Transforma vários números que em probabilidades que somam 1. É uma sigmoid gourmet
    layers.Dense(
        10,
        activations.softmax,
        kernel_initializer = initializers.RandomNormal()),

])

# Otimização e Métricas

Método usado para ensinar a rede neural: **Adam**

Decide como ajustar os pesos da rede e cada passo do treinamento. É como um *estilo de aprendizado*.
O que o Adam faz?

Imagine que você está descendo uma montanha de skate. Se você ganha velocidade numa direção, não para de repente - continua indo um pouco naquela direção.

**Momentum**  
Ajuda você a descer mais suave, evita oscilar demais e evita ficar preso em vales pequenos.  
  
**Adaptativo**  
Imagine caminhar com sapatos que se ajustam automaticamente. Se o terreno é difícil, ajuda mais. Se é fácil, ajuda menos.

O Adam:
 - dá passos maiores para partes que precisam aprender rápido
 - dá passos menores para partes que precisam de cuidado
 Cada peso da rede tem sua própria velocidade

 Em resumo, o Adam é uma mistura inteligente de Momentum + Adaptativo.

In [9]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.0015

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

# Lendo o Dataset
Imagens das categorias (Data)

In [11]:
from tensorflow.keras import utils

path = './Data'
batch_size = 100

train = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    subset = 'training',
    validation_split = 0.1, 
    seed = 1,
    image_size = (32, 32),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    subset = 'validation',
    validation_split = 0.1,
    seed = 1,
    image_size = (32, 32),
    batch_size = batch_size
)

Found 60000 files belonging to 1 classes.
Using 54000 files for training.
Found 60000 files belonging to 1 classes.
Using 6000 files for validation.
